# GPU Utilization

## Ingest code

## Setup

In [ ]:
import os 

import matplotlib.pylab as plt
import numpy as np

import tensorflow as tf
from tensorflow.keras import(
    Sequential,
    layers
)

In [ ]:
print(f'The version of TensorFlow is {tf.version.VERSION}')
gpu_support = 'Yes!' if tf.test.gpu_device_name() != '' else 'No!'
gpu_number = len(tf.config.list_physical_devices('GPU'))
print(f'Built with GPU support? {gpu_support}')
print(f'There are {gpu_number} GPUs')

device_name = tf.test.gpu_device_name()
if device_name == '':
    raise SystemError('GPU device not found')
else:
    print(f'Found GPU at {device_name}')

## Ingest Code

In [ ]:
%%writefile input.txt
gs://practical-ml-vision-book/images/california_fire1.jpg
gs://practical-ml-vision-book/images/california_fire2.jpg

In [ ]:
def read_jpeg(filename):
    img = tf.io.read_file(filename)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.reshape(img, [338, 600, 3])
    return img


ds = tf.data.TextLineDataset("input.txt").map(read_jpeg)
_, ax = plt.subplots(1, 2, figsize=(15, 10))
for idx, img in enumerate(ds):
    ax[idx].imshow(img.numpy())


## Adding a map function

In [ ]:
def to_grayscale(img):
    rows, cols, _ = img.shape
    result = np.zeros([rows, cols], dtype=np.float32)
    for row in range(rows):
        for col in range(cols):
            red = img[row][col][0]
            green = img[row][col][1]
            blue = img[row][col][2]
            c_linear = 0.2126 * red + 0.7152 * green + 0.0722 * blue
            if c_linear > 0.0031308:
                result[row][col] = 1.055 * pow(c_linear, 1/2.4) - 0.055
            else:
                result[row][col] = 12.92 * c_linear
    return result

In [ ]:
def to_grayscale_vct(img):
    wt = tf.constant([[0.2126], [0.7152], [0.0722]])
    c_linear = tf.matmul(img, wt)  # (height, width, 3) x (3,1) -> (height, width)
    gray = tf.where(
        c_linear > 0.0031308,
        1.055 * tf.pow(c_linear, 1 / 2.4) - 0.055,
        12.92 * c_linear,
    )
    
    return gray


In [ ]:
%%time
ds = tf.data.TextLineDataset('input.txt').repeat(10).map(read_jpeg)

overall = tf.constant([0.], dtype=tf.float32)
count = 0
for img in ds:
    gray = to_grayscale(img.numpy())
    m = tf.reduce_mean(gray, axis=[0, 1])
    overall += m
    count += 1
print(overall / count)

In [ ]:
%%time 
ds = tf.data.TextLineDataset('input.txt').repeat(10).map(read_jpeg).map(to_grayscale_vct)

overall = tf.constant([0.])
count = 0
for gray in ds:
    m = tf.reduce_mean(gray, axis=[0, 1])
    overall += m
    count += 1
print(overall/count)

## Batching

In [ ]:
class Grayscale(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(Grayscale, self).__init__(kwargs)

    def call(self, img):
        wt = tf.constant([[0.2126], [0.7152], [0.0722]])  # 3x1 matrix
        c_linear = tf.matmul(img, wt)  # (N, ht,wd,3) x (3x1) -> (N, ht, wd)
        gray = tf.where(
            c_linear > 0.0031308,
            1.055 * tf.pow(c_linear, 1 / 2.4) - 0.055,
            12.92 * c_linear,
        )
        return gray  # (N, ht, wd)


model = tf.keras.Sequential(
    [
        Grayscale(input_shape=(336, 600, 3)),
        tf.keras.layers.Lambda(
            lambda gray: tf.reduce_mean(gray, axis=[1, 2])
        ),  # note axis change
    ]
)
